In [1]:
import pandas as pd
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import math

### Lectura del fichero CSV

In [2]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

# Filtrado y construcción del dataframe binario
df_binario = data.iloc[:, 2:-2].apply(lambda x: x > 0).astype(int)

#Separar por vectores train y test
train_data = df_binario[data['Set'] == 'Train']
test_data = df_binario[data['Set'] == 'Test']

### Implementación de SWAP Test (2 vectores N columnas)
***Cuando subamos la cantidad de vectores hay que replantear cuantos wires (qbits) utilizar.*

Se calculan los qubits necesarios a partir de la dimensionalidad del vector (cantidad de columnas). El vector se define como:

$\mathbf{x} = (x_1, x_2, x_3, \ldots, x_n)$


Los qubits que vamos a necesitar para codificar (BasisEmbedding) son $ qubits = dim(\mathbf{x})$, ademas:
* Para cada dimension necesitamos un qbits ancilla
* para cada vector necesitamos un qubit por dimentsion

Si tenemos N vectores necesitaremos multiplicar N+1 veces los qubits requeridos para codificar N por cada vector y +1 para los qubits ancilla.



In [178]:
N = 5  # Numero de componentes
#train_vector = train_data.iloc[:1, :N].to_numpy()
#test_vector = test_data.iloc[:1, :N].to_numpy()
train_vector = [1,1,0,1,1]
test_vector = [0,0,1,0,0]
print(train_vector)
print(test_vector)
#tensor_product = np.kron(test_vector, train_vector)
#print("\n")
#print(tensor_product)

#Amplitude Embedding
qubit_number_AmplitudeEmbedding = int(np.ceil(np.log2(N))) if np.log2(N) > 0 else 1
qubits_totales_amplitude = 3*qubit_number_AmplitudeEmbedding

[1, 1, 0, 1, 1]
[0, 0, 1, 0, 0]


In [179]:
dev = qml.device("default.qubit", qubits_totales_amplitude, shots = 1500)
#dev_qiskit = qml.device("qiskit.aer", qubits_totales)
@qml.qnode(dev)
@qml.transforms.merge_amplitude_embedding
def circuit(test_vector, train_vector):
    qml.AmplitudeEmbedding(features=test_vector, wires=range(qubit_number_AmplitudeEmbedding,2*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    qml.AmplitudeEmbedding(features=train_vector, wires=range(2*qubit_number_AmplitudeEmbedding,3*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    ancillea = []
    for i in range(qubit_number_AmplitudeEmbedding):
        qml.Hadamard(i)
        anc = i
        ancillea.append(anc)
        first_state = i+qubit_number_AmplitudeEmbedding
        second_state = i+ 2*qubit_number_AmplitudeEmbedding
        qml.CSWAP(wires=[anc, first_state, second_state])
        qml.Hadamard(i)
    return qml.expval(qml.operation.Tensor(*[qml.PauliZ(i) for i in ancillea]))
    #qml.probs(wires = range(len(ancillea)))

estado_0 = circuit(test_vector, train_vector)
outliers = []
if not estado_0 > 0 :
    outliers.append(test_vector)
#dev.circuit.draw(output="mpl")
print(qml.draw(circuit)(test_vector, train_vector))
print(estado_0)

0: ──H──────────────────────╭●─────H───────────────────┤ ╭<Z@Z@Z>
1: ─────────────────────────│──────H─╭●─────H──────────┤ ├<Z@Z@Z>
2: ─────────────────────────│────────│──────H─╭●─────H─┤ ╰<Z@Z@Z>
3: ─╭AmplitudeEmbedding(M0)─├SWAP────│────────│────────┤         
4: ─├AmplitudeEmbedding(M0)─│────────├SWAP────│────────┤         
5: ─├AmplitudeEmbedding(M0)─│────────│────────├SWAP────┤         
6: ─├AmplitudeEmbedding(M0)─╰SWAP────│────────│────────┤         
7: ─├AmplitudeEmbedding(M0)──────────╰SWAP────│────────┤         
8: ─╰AmplitudeEmbedding(M0)───────────────────╰SWAP────┤         
M0 = 
[0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0.5+0.j
 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.